In [1]:
""" Ingest the inundation maps to earthegine
-------------------------------------------------------------------------------


Author: Rutger Hofste
Date: 20180808
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    SCRIPT_NAME (string) : Script name
    OUTPUT_VERSION (integer) : output version for ec2 and s3.
    GCS_INPUT_PATH (string) : input string for GCS
    EE_OUTPUT_PATH (string) : output path for Earthengine
    
Returns:

Result:
    Images in one imageCollection on earthengine

"""
SCRIPT_NAME = "Y2018M08D16_RH_Floods_Inundation_EE_V01"
OUTPUT_VERSION = 1

GCS_INPUT_PATH = "gs://aqueduct30_v01/Y2018M08D16_RH_Convertt_Geotiff_V01/output_V02/output_V02" #typo in pathname
EE_BASE_PATH = "projects/WRI-Aquaduct/floods/{}".format(SCRIPT_NAME)
EE_OUTPUT_PATH = EE_BASE_PATH+"/output_V{:02.0f}".format(OUTPUT_VERSION)
EC2_INPUT_PATH = "/volumes/data/Y2018M08D16_RH_Convertt_Geotiff_V01/output_V02" #for metadata 

EE_IMAGECOLLECTION_NAME = "inundation"

print("GCS_INPUT_PATH:" + GCS_INPUT_PATH +
      "\nEC2_INPUT_PATH:" + EC2_INPUT_PATH +
      "\nEE_OUTPUT_PATH:" + EE_OUTPUT_PATH)



GCS_INPUT_PATH:gs://aqueduct30_v01/Y2018M08D16_RH_Convertt_Geotiff_V01/output_V02/output_V02
EC2_INPUT_PATH:/volumes/data/Y2018M08D16_RH_Convertt_Geotiff_V01/output_V02
EE_OUTPUT_PATH:projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V01


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M08D16 UTC 16:28


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import os
import pickle
import subprocess
import ast

In [4]:
command = "earthengine create folder {}".format(EE_BASE_PATH)
response = subprocess.check_output(command,shell=True)
print(response)

b'Asset projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01 already exists\n'


In [5]:
command = "earthengine create folder {}".format(EE_OUTPUT_PATH)
response = subprocess.check_output(command,shell=True)
print(response)

b'Asset projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V01 already exists\n'


In [6]:
EE_IC_PATH = "{}/{}".format(EE_OUTPUT_PATH,EE_IMAGECOLLECTION_NAME)

command = "earthengine create collection {}".format(EE_IC_PATH)
response = subprocess.check_output(command,shell=True)
print(response)

b'Asset projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V01/inundation already exists\n'


In [7]:
def dict_to_command(d):
    # nodata value is specified separately since special tag required
    try:
        d.pop("inun__FillValue")
    except:
        pass    
    
    property_string = ""
    for key, value in d.items():
        
        
        if type(value) == int or type(value) == float:
            # value is number
            property_string = property_string + " -p '(number){}={}' ".format(key,value)
        else:
            # value is string or value is number in string format.        
            try:
                # Check if number in string format
                value = ast.literal_eval(value)
                property_string = property_string + " -p '(number){}={}' ".format(key,value)
            except:
                # if checknumber fails, is string. Clipping at 75 (max byte is 1024 in ee)
                value = value[:75] + (value[75:] and '..')
                property_string = property_string + " -p '(string){}={}' ".format(key,value) 
    return property_string

In [8]:
def create_ingest_command(input_path):
    with open(input_path, 'rb') as handle:
        dictje = pickle.load(handle)
        
    command = "earthengine upload image "    
    
    filename = dictje["filename"]
    filename_no_ext, ext = filename.split(".")
        
    asset_id = "{}/{}".format(EE_IC_PATH,filename_no_ext)
    command += "--asset_id={}".format(asset_id)    
    source_path = "{}/{}.tif".format(GCS_INPUT_PATH,filename_no_ext)    
    command += " {}".format(source_path)
    
    command += " --nodata_value=-9999 -p '(string)ingested_by=rutgerhofste' -p 'date"
    
    # properties_from_filename    
    command +=  dict_to_command(dictje["properties_from_filename"])

    # global attributes
    command +=  dict_to_command(dictje["global_attributes"])
    
    #variable attributes
    command +=  dict_to_command(dictje["variable_attributes"])
    
    return command

In [9]:
for root, dirs, files in os.walk(EC2_INPUT_PATH):
    for one_file in files:
        if one_file.endswith("pickle"):
            input_path = os.path.join(root,one_file)
            command = create_ingest_command(input_path)
            
            


In [10]:
print(command)

earthengine upload image --asset_id=projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V01/inundation/inuncoast_rcp4p5_wtsub_2030_rp1000_0_perc_50 gs://aqueduct30_v01/Y2018M08D16_RH_Convertt_Geotiff_V01/output_V02/output_V02/inuncoast_rcp4p5_wtsub_2030_rp1000_0_perc_50.tif --nodata_value=-9999 -p '(string)floodtype=inuncoast'  -p '(string)model=perc'  -p '(number)sea_level_rise_scenario=50'  -p '(number)returnperiod_decimal=0'  -p '(string)subsidence=wtsub'  -p '(string)climate=rcp4p5'  -p '(string)returnperiod=rp1000'  -p '(number)year=2030'  -p '(string)Conventions=CF-1.6'  -p '(string)history=Created by: $Id: coastal_inun.py 219 2017-11-13 11:12:22Z eilan_dk $,
     ..'  -p '(string)config_file=/p/1209884-aqueduct/Coastal_inun/settings_h6/coastal_inun.ini'  -p '(string)title=Aqueduct Coastal hazard layer'  -p '(string)references=http://floods.wri.org/'  -p '(string)institution=Deltares'  -p '(string)source=Global Tide and Surge Model'  -p '(string)project=Aq